In [3]:
import pandas as pd
import numpy as np
import wrds
from pandas.tseries.offsets import MonthEnd, QuarterEnd, QuarterBegin
from dateutil import parser
import time
#!pip install seaborn
import seaborn as sns

from numpy.random import randint

In [4]:
#conn = wrds.Connection()

1a

In [3]:
df1=pd.read_csv(r'C:\Users\Henry\OneDrive\Desktop\ACF\AcF351b\bond_yields_pt1.csv')

In [4]:
df1_1=pd.read_csv(r'C:\Users\Henry\OneDrive\Desktop\ACF\AcF351b\bond_yields_pt2.csv')

In [5]:
df2= df1.append(df1_1) #combined data as excel couldn't handle all the data at once

In [6]:
df2['trd_exctn_dt']= pd.to_datetime(df2['trd_exctn_dt'],format='%Y-%m-%d') #change date to datetime dtype

In [7]:
df2 = df2[df2['yld_pt'].notna()] #drop rows with no yields
df2 = df2.drop(columns=['bond_sym_id', 'company_symbol']) #drop bond symbol and company symbol columns.
df2 = df2.sort_values(by=['CUSIP_ID', 'trd_exctn_dt', 'trd_exctn_tm', 'yld_pt'], ascending=True) #sorted by cusip, date,time and yield
df2_ordered = df2
df2_ordered

,CUSIP_ID,yld_pt,trd_exctn_dt,trd_exctn_tm
891065,00077TAB0,6.525827,2015-01-05,14:27:45
891064,00077TAB0,6.608047,2015-01-05,14:27:45
891067,00077TAB0,6.786439,2015-01-06,10:23:34
891066,00077TAB0,6.842175,2015-01-06,10:23:34
891069,00077TAB0,6.897267,2015-02-17,11:24:15
...,...,...,...,...
3417,98934KAB6,2.971758,2017-11-30,17:23:00
3419,98934KAB6,3.009462,2017-12-04,09:27:55
3418,98934KAB6,3.210928,2017-12-04,09:27:55
3420,98934KAB6,2.977512,2017-12-19,12:22:54


In [8]:

#Dropped duplicate dates and kept last occurence
df3 = df2_ordered.drop_duplicates(subset=['CUSIP_ID', 'trd_exctn_dt'], keep='last')

In [9]:

#grouped by bond and each month contained sorted yields
Grouped = df3.groupby(['CUSIP_ID',
    pd.Grouper(key=('trd_exctn_dt'),freq='M')])
Grouped = Grouped.apply(lambda _df3: _df3.sort_values(by=['yld_pt'],ascending=True))

In [10]:
Grouped

CUSIP_ID    yld_pt trd_exctn_dt trd_exctn_tm
CUSIP_ID  trd_exctn_dt                                                      
00077TAB0 2015-01-31   891064  00077TAB0  6.608047   2015-01-05     14:27:45
                       891066  00077TAB0  6.842175   2015-01-06     10:23:34
          2015-02-28   891068  00077TAB0  7.002821   2015-02-17     11:24:15
          2015-03-31   101170  00077TAB0  6.816467   2015-03-31     14:59:18
          2015-04-30   101172  00077TAB0  6.029157   2015-04-24     12:10:55
...                                  ...       ...          ...          ...
98934KAB6 2017-11-30   3402    98934KAB6  2.936286   2017-11-22     09:27:49
                       3417    98934KAB6  2.971758   2017-11-30     17:23:00
                       3400    98934KAB6  3.036031   2017-11-14     15:54:28
          2017-12-31   3421    98934KAB6  2.960962   2017-12-19     12:22:55
                       3418    98934KAB6  3.210928   2017-12-04     09:27:55

[113786 rows x 4 columns]

In [11]:
Grouped = Grouped.reset_index(drop=True)

In [12]:
gp = Grouped

In [13]:
#ranked each yield by bond and sorted yield for each month
gp['rank'] = gp.groupby(['CUSIP_ID',
    pd.Grouper(key=('trd_exctn_dt'), freq='m')])['yld_pt'].rank()

In [14]:
gp

,CUSIP_ID,yld_pt,trd_exctn_dt,trd_exctn_tm,rank
0,00077TAB0,6.608047,2015-01-05,14:27:45,1.0
1,00077TAB0,6.842175,2015-01-06,10:23:34,2.0
2,00077TAB0,7.002821,2015-02-17,11:24:15,1.0
3,00077TAB0,6.816467,2015-03-31,14:59:18,1.0
4,00077TAB0,6.029157,2015-04-24,12:10:55,1.0
...,...,...,...,...,...
113781,98934KAB6,2.936286,2017-11-22,09:27:49,2.0
113782,98934KAB6,2.971758,2017-11-30,17:23:00,3.0
113783,98934KAB6,3.036031,2017-11-14,15:54:28,4.0
113784,98934KAB6,2.960962,2017-12-19,12:22:55,1.0


In [15]:
gp2 = gp.groupby(['CUSIP_ID',
    pd.Grouper(key=('trd_exctn_dt'), freq='m')])

In [16]:
gp2

In [17]:
#find median of the ranks for each group
med=gp2['rank'].transform('median') 

In [18]:
med

0         1.5
1         1.5
2         1.0
3         1.0
4         1.0
         ... 
113781    2.5
113782    2.5
113783    2.5
113784    1.5
113785    1.5
Name: rank, Length: 113786, dtype: float64

In [19]:
#rounds the median to whole number 
med2 = np.ceil(med) 

In [20]:
med2

0         2.0
1         2.0
2         1.0
3         1.0
4         1.0
         ... 
113781    3.0
113782    3.0
113783    3.0
113784    2.0
113785    2.0
Name: rank, Length: 113786, dtype: float64

In [21]:
#selects median observation and removes ranked column
final_1a = gp[gp['rank']==med2].drop('rank',axis=1)

In [22]:
#Removed trade execution time
final_1a= final_1a.drop(columns='trd_exctn_tm')

In [23]:
final_1a = final_1a.rename(columns={'CUSIP_ID':'cusip_id'})

In [24]:
final_1a

,cusip_id,yld_pt,trd_exctn_dt
1,00077TAB0,6.842175,2015-01-06
2,00077TAB0,7.002821,2015-02-17
3,00077TAB0,6.816467,2015-03-31
4,00077TAB0,6.029157,2015-04-24
6,00077TAB0,6.492385,2015-06-10
...,...,...,...
113764,98934KAB6,2.807131,2017-08-09
113771,98934KAB6,2.858460,2017-09-26
113777,98934KAB6,2.955087,2017-10-24
113782,98934KAB6,2.971758,2017-11-30


1a complete

1e

In [25]:
df=pd.read_csv(r'C:\Users\Henry\OneDrive\Desktop\ACF\AcF351b\acf_351b_python_data.csv')

In [26]:
df['maturity']= pd.to_datetime(df['maturity']) #change date to datetime dtype

In [27]:
df1e = pd.merge(final_1a,df[['cusip_id','maturity']],on='cusip_id', how='left')
df1e

,cusip_id,yld_pt,trd_exctn_dt,maturity
0,00077TAB0,6.842175,2015-01-06,2093-10-15
1,00077TAB0,7.002821,2015-02-17,2093-10-15
2,00077TAB0,6.816467,2015-03-31,2093-10-15
3,00077TAB0,6.029157,2015-04-24,2093-10-15
4,00077TAB0,6.492385,2015-06-10,2093-10-15
...,...,...,...,...
7766,98934KAB6,2.807131,2017-08-09,2023-11-15
7767,98934KAB6,2.858460,2017-09-26,2023-11-15
7768,98934KAB6,2.955087,2017-10-24,2023-11-15
7769,98934KAB6,2.971758,2017-11-30,2023-11-15


In [28]:
df1e['tm_to_mat_d'] = (df1e['maturity'] - df1e['trd_exctn_dt'])
df1e

,cusip_id,yld_pt,trd_exctn_dt,maturity,tm_to_mat_d
0,00077TAB0,6.842175,2015-01-06,2093-10-15,28772 days
1,00077TAB0,7.002821,2015-02-17,2093-10-15,28730 days
2,00077TAB0,6.816467,2015-03-31,2093-10-15,28688 days
3,00077TAB0,6.029157,2015-04-24,2093-10-15,28664 days
4,00077TAB0,6.492385,2015-06-10,2093-10-15,28617 days
...,...,...,...,...,...
7766,98934KAB6,2.807131,2017-08-09,2023-11-15,2289 days
7767,98934KAB6,2.858460,2017-09-26,2023-11-15,2241 days
7768,98934KAB6,2.955087,2017-10-24,2023-11-15,2213 days
7769,98934KAB6,2.971758,2017-11-30,2023-11-15,2176 days


In [29]:
df1e['tm_to_mat_Years'] = df1e.tm_to_mat_d / np.timedelta64(1, 'Y')
df1e


,cusip_id,yld_pt,trd_exctn_dt,maturity,tm_to_mat_d,tm_to_mat_Years
0,00077TAB0,6.842175,2015-01-06,2093-10-15,28772 days,78.775060
1,00077TAB0,7.002821,2015-02-17,2093-10-15,28730 days,78.660068
2,00077TAB0,6.816467,2015-03-31,2093-10-15,28688 days,78.545076
3,00077TAB0,6.029157,2015-04-24,2093-10-15,28664 days,78.479366
4,00077TAB0,6.492385,2015-06-10,2093-10-15,28617 days,78.350685
...,...,...,...,...,...,...
7766,98934KAB6,2.807131,2017-08-09,2023-11-15,2289 days,6.267069
7767,98934KAB6,2.858460,2017-09-26,2023-11-15,2241 days,6.135650
7768,98934KAB6,2.955087,2017-10-24,2023-11-15,2213 days,6.058988
7769,98934KAB6,2.971758,2017-11-30,2023-11-15,2176 days,5.957686


In [30]:
func = lambda x: round(x, 2)

In [31]:
df1e['tm_to_mat_Years'] = df1e['tm_to_mat_Years'].map(func)

In [32]:
df1e

,cusip_id,yld_pt,trd_exctn_dt,maturity,tm_to_mat_d,tm_to_mat_Years
0,00077TAB0,6.842175,2015-01-06,2093-10-15,28772 days,78.78
1,00077TAB0,7.002821,2015-02-17,2093-10-15,28730 days,78.66
2,00077TAB0,6.816467,2015-03-31,2093-10-15,28688 days,78.55
3,00077TAB0,6.029157,2015-04-24,2093-10-15,28664 days,78.48
4,00077TAB0,6.492385,2015-06-10,2093-10-15,28617 days,78.35
...,...,...,...,...,...,...
7766,98934KAB6,2.807131,2017-08-09,2023-11-15,2289 days,6.27
7767,98934KAB6,2.858460,2017-09-26,2023-11-15,2241 days,6.14
7768,98934KAB6,2.955087,2017-10-24,2023-11-15,2213 days,6.06
7769,98934KAB6,2.971758,2017-11-30,2023-11-15,2176 days,5.96


In [33]:
df1e = df1e.drop(columns='tm_to_mat_d')

In [34]:
df1em = df1e.sort_values(by='maturity')
df1em = df1em.rename(columns={'trd_exctn_dt': 'Date'})
df1em

,cusip_id,yld_pt,Date,maturity,tm_to_mat_Years
7670,957638AB5,5.820867,2015-03-20,2015-01-09,-0.19
7675,957638AB5,2.671614,2015-08-03,2015-01-09,-0.56
7674,957638AB5,8.673810,2015-07-21,2015-01-09,-0.53
7673,957638AB5,8.463005,2015-06-30,2015-01-09,-0.47
7672,957638AB5,5.444541,2015-05-27,2015-01-09,-0.38
...,...,...,...,...,...
2717,345370BS8,6.604350,2016-11-15,2097-05-15,80.50
2722,345370BS8,5.697198,2017-11-17,2097-05-15,79.49
2721,345370BS8,6.171229,2017-09-29,2097-05-15,79.63
2715,345370BS8,6.437057,2016-05-24,2097-05-15,80.98


In [35]:
#grate = df1e.groupby('cusip_id')

In [36]:
#for groups in grate:
   # print(groups)

In [37]:
#specify low_memory=False, i.e.,
#dffed=pd.read_csv(" XXX.csv", low_memory=False)

In [38]:
dffed =pd.read_csv(r'C:\Users\Henry\OneDrive\Desktop\ACF\AcF351b\1efeds.csv', low_memory=False)
dffed

,Date,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
0,25/10/2019,1.6727,1.6459,1.6344,1.6356,1.6473,1.6675,1.6945,1.7268,1.7632,...,2.2325,2.2618,2.2887,2.3132,2.3353,2.3551,2.3725,2.3877,2.4006,2.4113
1,24/10/2019,1.6482,1.6110,1.5953,1.5962,1.6098,1.6329,1.6630,1.6982,1.7369,...,2.1988,2.2277,2.2548,2.2799,2.3032,2.3247,2.3445,2.3625,2.3788,2.3936
2,23/10/2019,1.6433,1.6069,1.5916,1.5925,1.6058,1.6284,1.6580,1.6925,1.7305,...,2.1883,2.2173,2.2445,2.2699,2.2934,2.3152,2.3353,2.3537,2.3705,2.3856
3,22/10/2019,1.6492,1.6162,1.6026,1.6041,1.6173,1.6396,1.6686,1.7026,1.7401,...,2.1964,2.2253,2.2524,2.2776,2.3010,2.3225,2.3423,2.3604,2.3768,2.3916
4,21/10/2019,1.6523,1.6210,1.6092,1.6126,1.6279,1.6522,1.6833,1.7192,1.7585,...,2.2253,2.2543,2.2813,2.3063,2.3294,2.3507,2.3701,2.3877,2.4036,2.4178
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1451,08/01/2014,0.1735,0.4562,0.8956,1.3592,1.7897,2.1664,2.4863,2.7537,2.9756,...,3.9611,3.9852,4.0059,4.0240,4.0397,4.0534,4.0654,4.0760,4.0853,4.0936
1452,07/01/2014,0.1560,0.4100,0.8232,1.2735,1.7022,2.0845,2.4136,2.6909,2.9220,...,3.9417,3.9677,3.9906,4.0110,4.0292,4.0456,4.0605,4.0740,4.0864,4.0978
1453,06/01/2014,0.1591,0.4096,0.8252,1.2809,1.7156,2.1033,2.4363,2.7160,2.9480,...,3.9588,3.9852,4.0088,4.0300,4.0491,4.0666,4.0825,4.0971,4.1106,4.1231
1454,03/01/2014,0.1710,0.4139,0.8424,1.3105,1.7522,2.1417,2.4734,2.7506,2.9799,...,3.9884,4.0145,4.0375,4.0580,4.0763,4.0928,4.1078,4.1213,4.1337,4.1451


In [39]:
dffed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1456 entries, 0 to 1455
Data columns (total 31 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Date    1456 non-null   object 
 1   1       1456 non-null   float64
 2   2       1456 non-null   float64
 3   3       1456 non-null   float64
 4   4       1456 non-null   float64
 5   5       1456 non-null   float64
 6   6       1456 non-null   float64
 7   7       1456 non-null   float64
 8   8       1456 non-null   float64
 9   9       1456 non-null   float64
 10  10      1456 non-null   float64
 11  11      1456 non-null   float64
 12  12      1456 non-null   float64
 13  13      1456 non-null   float64
 14  14      1456 non-null   float64
 15  15      1456 non-null   float64
 16  16      1456 non-null   float64
 17  17      1456 non-null   float64
 18  18      1456 non-null   float64
 19  19      1456 non-null   float64
 20  20      1456 non-null   float64
 21  21      1456 non-null   float64
 22  

In [40]:
#dffed[(dffed['date'] > '2015-01-01') & (dffed['date'] < '2017-12-31')]

In [41]:
dffed['Date']= pd.to_datetime(dffed['Date'])

In [42]:
dffed

,Date,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
0,2019-10-25,1.6727,1.6459,1.6344,1.6356,1.6473,1.6675,1.6945,1.7268,1.7632,...,2.2325,2.2618,2.2887,2.3132,2.3353,2.3551,2.3725,2.3877,2.4006,2.4113
1,2019-10-24,1.6482,1.6110,1.5953,1.5962,1.6098,1.6329,1.6630,1.6982,1.7369,...,2.1988,2.2277,2.2548,2.2799,2.3032,2.3247,2.3445,2.3625,2.3788,2.3936
2,2019-10-23,1.6433,1.6069,1.5916,1.5925,1.6058,1.6284,1.6580,1.6925,1.7305,...,2.1883,2.2173,2.2445,2.2699,2.2934,2.3152,2.3353,2.3537,2.3705,2.3856
3,2019-10-22,1.6492,1.6162,1.6026,1.6041,1.6173,1.6396,1.6686,1.7026,1.7401,...,2.1964,2.2253,2.2524,2.2776,2.3010,2.3225,2.3423,2.3604,2.3768,2.3916
4,2019-10-21,1.6523,1.6210,1.6092,1.6126,1.6279,1.6522,1.6833,1.7192,1.7585,...,2.2253,2.2543,2.2813,2.3063,2.3294,2.3507,2.3701,2.3877,2.4036,2.4178
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1451,2014-08-01,0.1735,0.4562,0.8956,1.3592,1.7897,2.1664,2.4863,2.7537,2.9756,...,3.9611,3.9852,4.0059,4.0240,4.0397,4.0534,4.0654,4.0760,4.0853,4.0936
1452,2014-07-01,0.1560,0.4100,0.8232,1.2735,1.7022,2.0845,2.4136,2.6909,2.9220,...,3.9417,3.9677,3.9906,4.0110,4.0292,4.0456,4.0605,4.0740,4.0864,4.0978
1453,2014-06-01,0.1591,0.4096,0.8252,1.2809,1.7156,2.1033,2.4363,2.7160,2.9480,...,3.9588,3.9852,4.0088,4.0300,4.0491,4.0666,4.0825,4.0971,4.1106,4.1231
1454,2014-03-01,0.1710,0.4139,0.8424,1.3105,1.7522,2.1417,2.4734,2.7506,2.9799,...,3.9884,4.0145,4.0375,4.0580,4.0763,4.0928,4.1078,4.1213,4.1337,4.1451


In [43]:
#dffed.info()

In [44]:
#dfy= pd.merge(df1e,dffed[['Date','1']],on='Date', how='left')

In [45]:
df1em

,cusip_id,yld_pt,Date,maturity,tm_to_mat_Years
7670,957638AB5,5.820867,2015-03-20,2015-01-09,-0.19
7675,957638AB5,2.671614,2015-08-03,2015-01-09,-0.56
7674,957638AB5,8.673810,2015-07-21,2015-01-09,-0.53
7673,957638AB5,8.463005,2015-06-30,2015-01-09,-0.47
7672,957638AB5,5.444541,2015-05-27,2015-01-09,-0.38
...,...,...,...,...,...
2717,345370BS8,6.604350,2016-11-15,2097-05-15,80.50
2722,345370BS8,5.697198,2017-11-17,2097-05-15,79.49
2721,345370BS8,6.171229,2017-09-29,2097-05-15,79.63
2715,345370BS8,6.437057,2016-05-24,2097-05-15,80.98


In [56]:
upto = lambda x=30 if x<=30 else np.ceil(x)



SyntaxError: invalid syntax (<ipython-input-56-f520c2af275e>, line 1)

In [47]:
#def round(x):
    #if x<=30:
     #   x=30
    #else: x=np.ceil(x)

In [48]:
#print(round(31))

In [68]:
f = lambda x: 30 if x>=30 else 0 if x<0 else np.ceil(x)

In [69]:
df1em['coupyear']= df1em['tm_to_mat_Years'].map(f)

In [70]:
df1em

,cusip_id,yld_pt,Date,maturity,tm_to_mat_Years,coupyear
7670,957638AB5,5.820867,2015-03-20,2015-01-09,-0.19,0.0
7675,957638AB5,2.671614,2015-08-03,2015-01-09,-0.56,0.0
7674,957638AB5,8.673810,2015-07-21,2015-01-09,-0.53,0.0
7673,957638AB5,8.463005,2015-06-30,2015-01-09,-0.47,0.0
7672,957638AB5,5.444541,2015-05-27,2015-01-09,-0.38,0.0
...,...,...,...,...,...,...
2717,345370BS8,6.604350,2016-11-15,2097-05-15,80.50,30.0
2722,345370BS8,5.697198,2017-11-17,2097-05-15,79.49,30.0
2721,345370BS8,6.171229,2017-09-29,2097-05-15,79.63,30.0
2715,345370BS8,6.437057,2016-05-24,2097-05-15,80.98,30.0


In [1]:
g = df1em.groupby('coupyear')

NameError: name 'df1em' is not defined

In [2]:
for group in g:
    print(group)

NameError: name 'g' is not defined